<a href="https://colab.research.google.com/github/hfathie/hfvSPH_on_GPU/blob/main/hfv_sph_gpu_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%writefile hfv_sph_gpu.cu
#include <iostream>
#include <fstream>
#include <cmath>
#include <string>
#include <vector>
#include <sstream>
#include "myCppSPHLibs.h"
using namespace std;


int main(){

  // Reading params.GPU file.
  ifstream infile;
  infile.open("params.GPU");

  int N;
  float c_0, gammah, Rcld_in_pc, Rcld_in_cm, Mcld_in_g, muu, Mach;
  float grav_const_in_cgs, G;

  if (infile.is_open()){
    infile >> N;
    infile >> c_0;
    infile >> gammah;
    infile >> Rcld_in_pc;
    infile >> Rcld_in_cm;
    infile >> Mcld_in_g;
    infile >> muu;
    infile >> Mach;
    infile >> grav_const_in_cgs;
    infile >> G;
    infile.close();
  }else{
    cout << "File Not Found !!!" << endl;
  }

  float UnitRadius_in_cm = Rcld_in_cm;
  float UnitRadius_in_cm_2 = UnitRadius_in_cm*UnitRadius_in_cm;

  float UnitMass_in_g = Mcld_in_g;
  //--------------------- To avoid getting inf ! --------
  float UnitDensity_in_cgs = UnitMass_in_g / UnitRadius_in_cm_2;
  UnitDensity_in_cgs = UnitDensity_in_cgs / UnitRadius_in_cm;
  //-------------------------
  float Unit_u_in_cgs = grav_const_in_cgs * UnitMass_in_g / UnitRadius_in_cm;
  float Unit_P_in_cgs = UnitDensity_in_cgs * Unit_u_in_cgs;
  float unitVelocity = sqrt(grav_const_in_cgs * UnitMass_in_g / UnitRadius_in_cm);

  // Reading Hydra file.
  string fname = "GPU_IC_Anathpin_25k_RAND.csv";

  vector<vector<string>> content;
  vector<string> row;
  string line, word;
  
  fstream file (fname, ios::in);
  if(file.is_open())
  {
  while(getline(file, line))
  {
  row.clear();
  
  stringstream str(line);
  
  while(getline(str, word, ','))
  row.push_back(word);
  content.push_back(row);
  }
  }
  else
  cout<<"Could not open the file\n";

  // declaring the arrays.
  float *x, *d_x, *y, *d_y, *z, *d_z, *vx, *d_vx, *vy, *d_vy, *vz, *d_vz;
  float *mass, *d_mass, *h, *d_h, *hprevious, *d_hprevious, *rho, *d_rho;
  float *accx, *accy, *accz, *d_accx, *d_accy, *d_accz, *eps, *d_eps;
  float *P, *d_P, *csnd, *d_csnd, *divV, *d_divV, *curlV, *d_curlV;
  float *accx_sph, *accy_sph, *accz_sph, *d_accx_sph, *d_accy_sph, *d_accz_sph;
  float *accx_tot, *accy_tot, *accz_tot, *d_accx_tot, *d_accy_tot, *d_accz_tot;

  x = new float[N];
  y = new float[N];
  z = new float[N];

  vx = new float[N];
  vy = new float[N];
  vz = new float[N];

  accx = new float[N];
  accy = new float[N];
  accz = new float[N];

  mass = new float[N];
  h = new float[N];
  hprevious = new float[N];
  rho = new float[N];
  eps = new float[N];
  P = new float[N];
  csnd = new float[N];

  divV = new float[N];
  curlV = new float[N];

  accx_sph = new float[N];
  accy_sph = new float[N];
  accz_sph = new float[N];

  accx_tot = new float[N];
  accy_tot = new float[N];
  accz_tot = new float[N];

  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));
  cudaMalloc(&d_z, N*sizeof(float));

  cudaMalloc(&d_vx, N*sizeof(float));
  cudaMalloc(&d_vy, N*sizeof(float));
  cudaMalloc(&d_vz, N*sizeof(float));

  cudaMalloc(&d_accx, N*sizeof(float));
  cudaMalloc(&d_accy, N*sizeof(float));
  cudaMalloc(&d_accz, N*sizeof(float));

  cudaMalloc(&d_mass, N*sizeof(float));
  cudaMalloc(&d_h, N*sizeof(float));
  cudaMalloc(&d_hprevious, N*sizeof(float));
  cudaMalloc(&d_rho, N*sizeof(float));
  cudaMalloc(&d_eps, N*sizeof(float));
  cudaMalloc(&d_P, N*sizeof(float));
  cudaMalloc(&d_csnd, N*sizeof(float));

  cudaMalloc(&d_divV, N*sizeof(float));
  cudaMalloc(&d_curlV, N*sizeof(float));

  cudaMalloc(&d_accx_sph, N*sizeof(float));
  cudaMalloc(&d_accy_sph, N*sizeof(float));
  cudaMalloc(&d_accz_sph, N*sizeof(float));

  cudaMalloc(&d_accx_tot, N*sizeof(float));
  cudaMalloc(&d_accy_tot, N*sizeof(float));
  cudaMalloc(&d_accz_tot, N*sizeof(float));

  // 0  1  2  3   4   5   6  7          8
  // x, y, z, vx, vy, vz, m, hprevious, eps

  // Initialize x, y, and z on the Host.
  for(int i = 0; i < N; i++){
    x[i] = stof(content[i][0]);
    y[i] = stof(content[i][1]);
    z[i] = stof(content[i][2]);

    vx[i] = stof(content[i][3]);
    vy[i] = stof(content[i][4]);
    vz[i] = stof(content[i][5]);

    mass[i] = stof(content[i][6]);
    hprevious[i] = stof(content[i][7]);
    eps[i] = stof(content[i][8]);
    h[i] = 10.0f; // place holder.
    rho[i] = 11.0f; // place holder.
    P[i] = 12.0f; // placeholder.
    csnd[i] = 13.0f; // placeholder.

    divV[i] = 14.0f; // placeholder.
    curlV[i] = 15.0f; // placeholder.

    accx[i] = 0.0f;
    accy[i] = 0.0f;
    accz[i] = 0.0f;

    accx_sph[i] = 0.0f;
    accy_sph[i] = 0.0f;
    accz_sph[i] = 0.0f;

    accx_tot[i] = 0.0f;
    accy_tot[i] = 0.0f;
    accz_tot[i] = 0.0f;
  }

  // Copy from Host to Device.
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_z, z, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_vx, vx, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_vy, vy, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_vz, vz, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_accx, accx, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_accy, accy, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_accz, accz, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_mass, mass, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_h, h, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_hprevious, hprevious, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_rho, rho, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_eps, eps, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_P, P, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_csnd, csnd, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_divV, divV, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_curlV, curlV, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_accx_sph, accx_sph, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_accy_sph, accy_sph, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_accz_sph, accz_sph, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_accx_tot, accx_tot, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_accy_tot, accy_tot, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_accz_tot, accz_tot, N*sizeof(float), cudaMemcpyHostToDevice);

  int blockSize = 64; // number of threads in a block
  int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks in a grid
  
  const int Nngb = 64;
  const int Ndown = Nngb - 5;
  const int Nup = Nngb + 5;
  const float coeff = 0.001; // used for smoothing length.

  const float visc_alpha = 1.0f;

  float mH = 1.6726e-24; // gram
  float kB = 1.3807e-16; // cm2 g s-2 K-1
  float mH2 = muu * mH;

  float kBmH2 = kB/mH2;

  float T_cld = 10.0f;
  float T_ps = 10.0f;
  float T_0 = 10.0f;

  const float my_pi = 3.141592f;

  float t = 0.0f;
  float dt = 2e-4;
  float tEnd = 3.0f;
  float Nt = ceil(tEnd/dt) + 1;

  //-----------------------------------------------
  //-------------- Smoothing Length ---------------
  //-----------------------------------------------

  smoothing_h<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_h, d_hprevious,
                                       N, Ndown, Nup, coeff);
  cudaDeviceSynchronize();
  
  //-----------------------------------------------
  //----------------- getDensity ------------------
  //-----------------------------------------------
  
  getDensity<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_mass,
                                      d_rho, d_h, my_pi, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //------------------ getAcc_g -------------------
  //-----------------------------------------------

  acc_g<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_eps, d_accx, d_accy, d_accz,
                                 d_mass, G, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //---------------- getPressure ------------------
  //-----------------------------------------------

  getPressure<<<gridSize, blockSize>>>(d_P, d_rho, T_cld, T_ps, T_0, kBmH2,
                                       UnitDensity_in_cgs, Unit_P_in_cgs,
                                       gammah, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //----------------- getCsound -------------------
  //-----------------------------------------------

  getCsound<<<gridSize, blockSize>>>(d_csnd, d_rho, T_cld, T_ps, T_0, kBmH2,
                                       UnitDensity_in_cgs, unitVelocity,
                                       gammah, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //----------------- div_curlV -------------------
  //-----------------------------------------------

  div_curlVel<<<gridSize, blockSize>>>(d_divV, d_curlV, d_x, d_y, d_z, d_vx, d_vy, d_vz,
                                       d_rho, d_mass, d_h, my_pi, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //------------------ acc_sph --------------------
  //-----------------------------------------------

  acc_sph<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_vx, d_vy, d_vz, d_h, d_csnd, d_rho,
                                   d_divV, d_curlV, d_mass, d_P, d_accx_sph, d_accy_sph,
                                   d_accz_sph, my_pi, visc_alpha, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //------------------ acc_tot --------------------
  //-----------------------------------------------
  
  acc_g_sph<<<gridSize, blockSize>>>(d_accx_tot, d_accy_tot, d_accz_tot,
                                     d_accx, d_accy, d_accz,
                                     d_accx_sph, d_accy_sph, d_accz_sph,
                                     N);
  cudaDeviceSynchronize();


  // LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? 
  // LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? 
  // LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? LOOP ? 
  
  int counter = 0;

  while(t < tEnd){

    /*
    cudaMemcpy(h, d_h, N*sizeof(float), cudaMemcpyDeviceToHost);
    for(int i = 0; i < 5; i++){
      cout << "h (before): " << h[i] << endl;
    }
    cout << "============" << endl;
    */
    

    //****************** velocity evolution *******************
    v_evolve<<<gridSize, blockSize>>>(d_vx, d_vy, d_vz, d_accx, d_accy, d_accz,
                                      dt, N);
    cudaDeviceSynchronize();
    
    //****************** position evolution *******************
    r_evolve<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_vx, d_vy, d_vz, dt, N);
    cudaDeviceSynchronize();

    //****************** Smoothing Length *********************
    smoothing_h<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_h, d_hprevious,
                                        N, Ndown, Nup, coeff);
    cudaDeviceSynchronize();

    //****************** getDensity ***********************
    getDensity<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_mass,
                                        d_rho, d_h, my_pi, N);
    cudaDeviceSynchronize();

    //****************** getAcc_g *************************
    acc_g<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_eps, d_accx, d_accy, d_accz,
                                  d_mass, G, N);
    cudaDeviceSynchronize();

    //****************** getPressure **********************
    getPressure<<<gridSize, blockSize>>>(d_P, d_rho, T_cld, T_ps, T_0, kBmH2,
                                        UnitDensity_in_cgs, Unit_P_in_cgs,
                                        gammah, N);
    cudaDeviceSynchronize();

    //****************** getCsound ************************
    getCsound<<<gridSize, blockSize>>>(d_csnd, d_rho, T_cld, T_ps, T_0, kBmH2,
                                        UnitDensity_in_cgs, unitVelocity,
                                        gammah, N);
    cudaDeviceSynchronize();

    //****************** div_curlV ************************
    div_curlVel<<<gridSize, blockSize>>>(d_divV, d_curlV, d_x, d_y, d_z, d_vx, d_vy, d_vz,
                                        d_rho, d_mass, d_h, my_pi, N);
    cudaDeviceSynchronize();

    //****************** acc_sph ************************** 
    acc_sph<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_vx, d_vy, d_vz, d_h, d_csnd, d_rho,
                                    d_divV, d_curlV, d_mass, d_P, d_accx_sph, d_accy_sph,
                                    d_accz_sph, my_pi, visc_alpha, N);
    cudaDeviceSynchronize();

    //****************** acc_tot **************************
    acc_g_sph<<<gridSize, blockSize>>>(d_accx_tot, d_accy_tot, d_accz_tot,
                                      d_accx, d_accy, d_accz,
                                      d_accx_sph, d_accy_sph, d_accz_sph,
                                      N);
    cudaDeviceSynchronize();

    //****************** velocity evolution *******************
    v_evolve<<<gridSize, blockSize>>>(d_vx, d_vy, d_vz, d_accx, d_accy, d_accz,
                                      dt, N);
    cudaDeviceSynchronize();

    t += dt;
    counter ++;

    if(!(counter % 100)){
      cout << "t = " << t << endl;
    }

    if(t >= 1.0f){
      cudaMemcpy(x, d_x, N*sizeof(float), cudaMemcpyDeviceToHost);
      cudaMemcpy(y, d_y, N*sizeof(float), cudaMemcpyDeviceToHost);
      cudaMemcpy(z, d_z, N*sizeof(float), cudaMemcpyDeviceToHost);

      ofstream outfile("Gout.csv");
      
      for(int i = 0; i < N; i++){
        outfile << x[i] << "," << y[i] << "," << z[i] << endl;
      }
      break;
    }
    
  }


  delete[] x; delete[] y; delete[] z; delete[] vx; delete[] vy; delete[] vz;
  delete[] mass; delete[] h; delete[] hprevious; delete[] rho;
  delete[] accx; delete[] accy; delete[] accz; delete[] eps;
  delete[] P; delete[] csnd; delete[] divV; delete[] curlV;
  delete[] accx_sph; delete[] accy_sph; delete[] accz_sph;
  delete[] accx_tot; delete[] accy_tot; delete[] accz_tot;

  cudaFree(d_x); cudaFree(d_y); cudaFree(d_z);
  cudaFree(d_vx); cudaFree(d_vy); cudaFree(d_vz);
  cudaFree(d_mass); cudaFree(d_h); cudaFree(d_hprevious);
  cudaFree(d_rho); cudaFree(d_accx); cudaFree(d_accy); cudaFree(d_accz);
  cudaFree(d_P); cudaFree(d_csnd); cudaFree(d_divV); cudaFree(d_curlV);
  cudaFree(d_accx_sph); cudaFree(d_accy_sph); cudaFree(d_accz_sph);
  cudaFree(d_accx_tot); cudaFree(d_accy_tot); cudaFree(d_accz_tot);

}

Overwriting hfv_sph_gpu.cu


In [14]:
%%shell
nvcc hfv_sph_gpu.cu

In [15]:
%%shell
./a.out

t = 0.02
t = 0.0399999
t = 0.0599999
t = 0.0800002
t = 0.100001
t = 0.120001
t = 0.140001
t = 0.160002
t = 0.180002
t = 0.200002
t = 0.220003
t = 0.240003
t = 0.260003
t = 0.280004
t = 0.300004
t = 0.320004
t = 0.340005
t = 0.360005
t = 0.380005
t = 0.400006
t = 0.420006
t = 0.440006
t = 0.460007
t = 0.480007
t = 0.500007
t = 0.520005
t = 0.540002
t = 0.559999
t = 0.579997
t = 0.599994
t = 0.619991
t = 0.639989
t = 0.659986
t = 0.679983
t = 0.699981
t = 0.719978
t = 0.739976
t = 0.759973
t = 0.77997
t = 0.799968
t = 0.819965
t = 0.839962
t = 0.85996
t = 0.879957
t = 0.899954
t = 0.919952
t = 0.939949
t = 0.959947
t = 0.979944
t = 0.999941


In [4]:
%%shell
nvprof ./a.out

==183== NVPROF is profiling process 183, command: ./a.out
t = 0.0002
t = 0.0004
t = 0.0006
t = 0.0008
t = 0.001
==183== Profiling application: ./a.out
==183== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   90.43%  10.3512s         6  1.72520s  1.70344s  1.79128s  smoothing_h(float*, float*, float*, float*, float*, int, int, int, float)
                    3.48%  398.68ms         6  66.446ms  59.549ms  90.817ms  acc_sph(float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float, float, int)
                    2.76%  315.65ms         6  52.608ms  43.707ms  55.790ms  acc_g(float*, float*, float*, float*, float*, float*, float*, float*, float, int)
                    2.26%  258.93ms         6  43.155ms  37.229ms  45.267ms  div_curlVel(float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float, int)
          

In [1]:
%%shell
nvidia-smi

Tue Nov 22 20:05:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   27C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------